In [ ]:
!pip install uv
!uv init

In [ ]:
!python3 --version

In [ ]:
# --system : 실제 노트북 커널이 패키지를 알게끔 함

!uv pip install --system --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
!uv pip install --system huggingface_hub pyarrow datasets peft bitsandbytes
!uv pip install --system "trl==0.11.4" "transformers==4.45.2" "accelerate==0.34.2" "peft==0.13.2" "bitsandbytes>=0.45.0" "numpy<2.2"
!uv pip install --system -U wandb
!uv pip install --system -U transformers accelerate peft
!uv pip install --system -U transformers trl peft bitsandbytes

In [ ]:
import torch
import numpy as np
import transformers

print(f"파이토치 버전: {torch.__version__}")
print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.version}")
print(f"NumPy version: {np.__version__}")
print(f"transformers version: {transformers.__version__}")

if torch.cuda.is_available() :
    capability = torch.cuda.get_device_capability()
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
    print(f"Compute Capability: {capability[0]}.{capability[1]}")
    print(f"BF16 지원 여부: {torch.cuda.is_bf16_supported()}")
else:
    print("CUDA를 사용할 수 없습니다.")

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    TrainingArguments,
)
from datasets import Dataset
import os, torch, json, wandb, subprocess
from sklearn.model_selection import train_test_split
import torch.nn as nn
from peft import (
    get_peft_model,
    LoraConfig, 
    TaskType,
    prepare_model_for_kbit_training
)
from accelerate import Accelerator, notebook_launcher

In [ ]:
from huggingface_hub import snapshot_download

save_path = f"/kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

snapshot_download(
    repo_id="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    local_dir=save_path,
    local_dir_use_symlinks=False,  # 실제 파일을 해당 경로에 직접 복사/다운로드
    max_workers=8 # 병렬 다운로드 속도 향상
)

notebook_launcher <- X

In [ ]:
script = '''#!/usr/bin/env python3

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TZ'] = 'Asia/Seoul'
time.tzset()

import torch
import wandb
from datetime import datetime
from functools import partial
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

# ============================================
# 1. HuggingFace 로그인
# ============================================
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("hf_token")
    if hf_token:
        login(token=hf_token)
        print("✅ Logged in to HuggingFace")
except Exception as e:
    print(f"⚠️ Could not retrieve HF secret: {e}")
    hf_token = None

PROJECT_NAME="library-faq-finetune"

os.environ["WANDB_PROJECT"] = PROJECT_NAME # 프로젝트 이름
os.environ["WANDB_RUN_ID"] = "v1" # 노트북 고유 ID
os.environ["WANDB_RESUME"] = "allow" # 해당 노트북에서 학습을 이어서 진행할 것인지

try:
    wandb_token = user_secrets.get_secret("wandb")
    if wandb_token:
        wandb.login(key=wandb_token)
        # timeout 늘리기
        wandb.init(
            project=os.environ["WANDB_PROJECT"],
            id=os.environ["WANDB_RUN_ID"],
            resume=os.environ["WANDB_RESUME"],
            name=datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 대시보드에 표시될 이름
        )
        print("✅ Logged in to W&B")
except Exception as e:
    print(f"⚠️ W&B login failed: {e}")
    os.environ["WANDB_DISABLED"] = "true"

# ============================================
# 2. 모델 & 토크나이저 먼저 로드 (generate_prompts에서 사용)
# ============================================
model_id = "/kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# ============================================
# 3. 데이터셋 로드
# ============================================
print("Loading dataset...")
dataset = load_dataset("JungSeong2/library-qa", token=hf_token)

train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

print(f"✅ Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

# ============================================
# 4. Formatting Function (토크나이저가 이미 로드됨)
# ============================================
def generate_prompts(examples):
    """배치 처리를 위한 함수"""
    question = examples["question"]
    answer = examples["answer"]
    context = examples["DES"]
    
    system_message = (
        "당신은 도서관 운영에 대한 전문적인 지식을 가진 인공지능 사서입니다. "
        "제공된 [도서관 정보]를 바탕으로 사용자의 질문에 정확하고 친절하게 답하십시오. "
        "정보에 없는 내용은 함부로 추측하지 말고 정중히 확인이 어렵다고 답하세요."
    )

    user_content = (
        f"### [도서관 정보]\\n{context}\\n\\n"
        f"### [질문]\\n{question}\\n\\n"
        f"### [지시 사항]\\n"
        f"1. 친절한 말투로 규정에 근거하여 답변할 것.\\n"
        f"2. 3문단 이내로 답변할 것.\\n"
        f"3. 답변 끝에 지시 사항을 반복하지 말 것."
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": answer}
    ]

    # apply_chat_template 적용
    full_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=False
    )
    
    return full_prompt

# ============================================
# 5. Config
# ============================================
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# ✅ EXAONE 모델용: o_proj 사용 (out_proj 아님)
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"]  # ✅ 수정
)

# ============================================
# 6. 모델 로드 및 PEFT 적용
# ============================================
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    trust_remote_code=True,
    device_map="auto"
)

if hasattr(model, "transformer") and hasattr(model.transformer, "wte"):
    model.get_input_embeddings = lambda: model.transformer.wte

print("Applying PEFT...")
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# ============================================
# 7. Training Arguments
# ============================================
training_args = SFTConfig(
    output_dir="./SFT_single_gpu",
    num_train_epochs=1,
    
    # 배치 설정
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    
    # 최적화
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    max_grad_norm=1.0,
    
    # 정밀도
    fp16=False,
    bf16=True,
    
    # Gradient checkpointing
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    
    # 데이터 로딩
    dataloader_num_workers=0,
    
    # SFT 설정
    max_length=1024, 
    packing=False,  # ✅ 일단 False로 (안정성)
    
    # Logging & Saving
    logging_steps=10,
    logging_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    
    # Evaluation (선택사항)
    eval_strategy="steps",
    eval_steps=100,
    
    # Reporting
    report_to="wandb"
)

# ============================================
# 8. Trainer 생성 및 학습
# ============================================
print("Creating trainer...")
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # ✅ validation 추가
    processing_class=tokenizer,  # ✅ processing_class 사용
    formatting_func=generate_prompts,
)

# 저장 패치
torch.load = partial(torch.load, weights_only=False)

print("🚀 Starting training...")
trainer.train()

# ============================================
# 9. 모델 저장
# ============================================
print("Saving model...")
trainer.save_model("./SFT_single_gpu/final")
tokenizer.save_pretrained("./SFT_single_gpu/final")
print("✅ Training Complete!")
'''

with open("train_single.py", "w") as f:
    f.write(script)

print("✅ Script created: train_single.py")

In [ ]:
!python3 train_single.py

In [11]:
script = '''#!/usr/bin/env python3

import os
import torch
import wandb
import time
from datetime import datetime
from functools import partial
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from accelerate import Accelerator

# ============================================
# 1. 초기화 및 환경 설정
# ============================================
accelerator = Accelerator()
# 시간을 서울 시간으로 설정 (메인 프로세스에서만)
if accelerator.is_main_process:
    os.environ['TZ'] = 'Asia/Seoul'
    time.tzset()

# HuggingFace 로그인
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("hf_token")
    if hf_token:
        login(token=hf_token)
except Exception:
    hf_token = None

# W&B 설정 (메인 프로세스에서만 초기화)
PROJECT_NAME = "library-faq-finetune"
if accelerator.is_main_process:
    try:
        wandb_token = user_secrets.get_secret("wandb")
        if wandb_token:
            wandb.login(key=wandb_token)
            wandb.init(
                project=PROJECT_NAME,
                name=f"multi-gpu-{datetime.now().strftime('%m%d-%H%M')}"
            )
    except:
        os.environ["WANDB_DISABLED"] = "true"
else:
    os.environ["WANDB_DISABLED"] = "true"

# ============================================
# 2. 모델 & 토크나이저 로드
# ============================================
model_id = "/kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# ============================================
# 3. 데이터셋 로드
# ============================================
print("Loading dataset...")
dataset = load_dataset("JungSeong2/library-qa", token=hf_token)

train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

print(f"✅ Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

def generate_prompts(examples):
    """배치 처리를 위한 함수"""
    question = examples["question"]
    answer = examples["answer"]
    context = examples["DES"]
    
    system_message = (
        "당신은 도서관 운영에 대한 전문적인 지식을 가진 인공지능 사서입니다. "
        "제공된 [도서관 정보]를 바탕으로 사용자의 질문에 정확하고 친절하게 답하십시오. "
        "정보에 없는 내용은 함부로 추측하지 말고 정중히 확인이 어렵다고 답하세요."
    )

    user_content = (
        f"### [도서관 정보]\\n{context}\\n\\n"
        f"### [질문]\\n{question}\\n\\n"
        f"### [지시 사항]\\n"
        f"1. 친절한 말투로 규정에 근거하여 답변할 것.\\n"
        f"2. 3문단 이내로 답변할 것.\\n"
        f"3. 답변 끝에 지시 사항을 반복하지 말 것."
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": answer}
    ]

    # apply_chat_template 적용
    full_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=False
    )
    
    return full_prompt

# ============================================
# 4. Config & 모델 로드 (Multi-GPU 최적화)
# ============================================
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    # EXAONE-3.5-2.4B 기준 타겟 모듈 확인 필요
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"] 
)

# device_map 설정 변경: 현재 프로세스의 GPU 번호를 명시적으로 할당
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    trust_remote_code=True,
    device_map={"": accelerator.process_index} # 핵심: 현재 프로세스 인덱스 할당
)

if hasattr(model, "transformer") and hasattr(model.transformer, "wte"):
    model.get_input_embeddings = lambda: model.transformer.wte

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# ============================================
# 5. Training Arguments (DDP 최적화)
# ============================================
training_args = SFTConfig(
    output_dir="./SFT_multi_gpu",
    num_train_epochs=1,
    per_device_train_batch_size=1, # 각 GPU당 배치 사이즈
    gradient_accumulation_steps=4, # GPU가 2개이므로 4*2=8 효과
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    bf16=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    logging_steps=5,
    save_strategy="no", # 학습 종료 후 수동 저장 권장
    eval_strategy="no",
    report_to="wandb" if accelerator.is_main_process else "none",
    # DDP 설정
    ddp_find_unused_parameters=False,
    max_length=1024,
    run_name=f"{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}"
)

# ============================================
# 6. Trainer 생성 및 학습
# ============================================
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    processing_class=tokenizer,
    formatting_func=generate_prompts,
)

# 학습 시작
trainer.train()

# 메인 프로세스에서만 모델 저장
if accelerator.is_main_process:
    trainer.save_model("./SFT_multi_gpu/final")
    tokenizer.save_pretrained("./SFT_multi_gpu/final")
    print("🚀 All processes finished! Model saved.")
    wandb.finish()
'''

with open("train_multi_gpu.py", "w") as f:
    f.write(script)

print("✅ Script created: train_multi_gpu.py")

✅ Script created: train_multi_gpu.py


In [12]:
!accelerate launch --multi_gpu --num_processes 2 --num_machines 1 --mixed_precision bf16 train_multi_gpu.py

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
The repository /kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at /kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct .
 You can inspect the repository content at https://hf.co//kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials f

In [16]:
from peft import PeftModel
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    TrainingArguments,
)
import torch

# 1. 학습 완료 후 저장된 어댑터 경로
base_model_path = "/kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
adapter_path = "/kaggle/working/SFT_multi_gpu/final"
repo_id = "JungSeong2/library-QA-Adapter" # 허깅페이스 저장소 이름
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # 4bit 할 것이냐
    bnb_4bit_compute_dtype=torch.bfloat16, #bfloat16 or float16
    bnb_4bit_quant_type="nf4", # nf4 or fp4
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto"
)

if hasattr(model, "transformer") and hasattr(model.transformer, "wte"):
    model.get_input_embeddings = lambda: model.transformer.wte

# 2. 토크나이저와 어댑터 모델 로드 (또는 학습 중인 trainer.model 사용)
model = PeftModel.from_pretrained(model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# 3. 어댑터 가중치만 업로드
model.push_to_hub(
    repo_id=repo_id,
    commit_message="[1 Epoch] FineTuned with EXAONE-3.5-2.4B-Instruct",
    revision="v1.0" # v2.0 브랜치에 해당 어댑터를 업로드
)

tokenizer.push_to_hub(
    repo_id,
    commit_message="[1 Epoch] FineTuned with EXAONE-3.5-2.4B-Instruct",
    revision="v1.0"
)

The repository /kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at /kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct .
 You can inspect the repository content at https://hf.co//kaggle/working/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Loading weights:   0%|          | 0/272 [00:00<?, ?it/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/JungSeong2/library-QA-Adapter/commit/e3ba893494db6f29bf4fa83fef18945620436c9c', commit_message='[1 Epoch] FineTuned with EXAONE-3.5-2.4B-Instruct', commit_description='', oid='e3ba893494db6f29bf4fa83fef18945620436c9c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/JungSeong2/library-QA-Adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='JungSeong2/library-QA-Adapter'), pr_revision=None, pr_num=None)